## Made by:
# Bhavneek Singh
# 2020EE10482
## IIT Delhi

In [1]:
!pip install -q -U torch torchvision torchaudio transformers accelerate optimum packaging ninja huggingface-hub hf_transfer langsmith langchain ctransformers[cuda] sentence-transformers chromadb

In [2]:
!wget https://chaabiv2.s3.ap-south-1.amazonaws.com/hiring/bigBasketProducts.csv

--2023-11-30 13:16:29--  https://chaabiv2.s3.ap-south-1.amazonaws.com/hiring/bigBasketProducts.csv
Resolving chaabiv2.s3.ap-south-1.amazonaws.com (chaabiv2.s3.ap-south-1.amazonaws.com)... 16.12.40.58, 52.219.158.194, 52.219.160.18, ...
Connecting to chaabiv2.s3.ap-south-1.amazonaws.com (chaabiv2.s3.ap-south-1.amazonaws.com)|16.12.40.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16739247 (16M) [text/csv]
Saving to: ‘bigBasketProducts.csv.1’

bigBasketProducts.c 100%[===================>]  15.96M  22.3MB/s    in 0.7s    

2023-11-30 13:16:30 (22.3 MB/s) - ‘bigBasketProducts.csv.1’ saved [16739247/16739247]



In [3]:
from langchain.llms import CTransformers
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [4]:
loader = CSVLoader(file_path="/content/bigBasketProducts.csv")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=2100, chunk_overlap=100)
data_chunks = text_splitter.split_documents(data)

In [5]:
print(len(data_chunks))
data_chunks[0]

27573


Document(page_content='index: 1\nproduct: Garlic Oil - Vegetarian Capsule 500 mg\ncategory: Beauty & Hygiene\nsub_category: Hair Care\nbrand: Sri Sri Ayurveda\nsale_price: 220\nmarket_price: 220\ntype: Hair Oil & Serum\nrating: 4.1\ndescription: This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/', metadata={'source': '/content/bigBasketProducts.csv', 'row': 0})

In [6]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Mistral-7B-Claude-Chat-GGUF mistral-7b-claude-chat.Q4_K_M.gguf

/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Claude-Chat-GGUF/snapshots/11eef398811ac0c9b0b2431485a7025f03c723c7/mistral-7b-claude-chat.Q4_K_M.gguf


In [7]:
config = {'max_new_tokens': 100, 'repetition_penalty': 1.1, 'temperature':0.3, 'gpu_layers':50, 'context_length':4096}
llm = CTransformers(model='/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Claude-Chat-GGUF/snapshots/11eef398811ac0c9b0b2431485a7025f03c723c7/mistral-7b-claude-chat.Q4_K_M.gguf'
      , model_type='mistral', config=config)

In [8]:
prompt_template = """A chat between a curious user and an artificial intelligence assistant.
The assistant gives helpful, detailed, and polite answers to the user's questions. Take help
from the given context. Don't make up information, just say "Sorry, I don't know".
Give short responses.
CONTEXT: {context}
USER: {question}
ASSISTANT:
 """

In [9]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [10]:
# chroma = Chroma.from_documents(
#     data_chunks,
#     embeddings,
#     persist_directory="/content/"
# )

In [11]:
# from google.colab import drive
# drive.mount('/content/drive')

In [12]:
# source_file = '/content/chroma.sqlite3'
# destination_path = '/content/drive/MyDrive/'
# !cp {source_file} {destination_path}

In [13]:
file_url = 'https://drive.google.com/uc?id=19ISXmlVT_sEXYcJKg9nLvtX5ujA7kwxa'
destination_path = '/content/'
!gdown $file_url -O $destination_path

Downloading...
From: https://drive.google.com/uc?id=19ISXmlVT_sEXYcJKg9nLvtX5ujA7kwxa
To: /content/chroma.sqlite3
100% 250M/250M [00:02<00:00, 112MB/s]


In [14]:
chroma = Chroma(persist_directory="/content/", embedding_function=embeddings)

In [15]:
retriever = chroma.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 5}
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [16]:
question = "Suggest some products from Garnier"
response = qa_chain({"query": question})
print(response['result'])

1. Garnier Light Complete Lotion - This lotion is enriched with Yuzu Lemon Essence and Vitamin C to lighten dark spots and brighten skin tone while moisturizing the skin. It has UVA/UVB filters that protect the skin from harmful sun rays.

2. Garnier Light Complete Sheet Mask - This sheet mask is infused with lemon and vitamin C to instantly hydrate, brighten and reduce spots


In [17]:
!pip -q install flask-ngrok

In [18]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def llm_response():
    try:
        question = request.args.get('question', '')
        response = qa_chain({"query": question})
        print(response['result'])
        return jsonify({'response': response['result']})
    except Exception as e:
        return jsonify({'error': str(e)})
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://92e9-34-124-216-252.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 13:20:48] "GET /?question=Suggest_some_beauty_products HTTP/1.1" 200 -


1. For glowing skin, try using a facial cleanser and toner every morning and evening to remove dirt and oil from the pores. Look for products that contain natural ingredients like aloe vera, chamomile or green tea extracts.

2. Exfoliate your face once or twice a week with a gentle scrub to slough off dead skin cells and reveal brighter skin underneath. Use circular motions and avoid harsh scrubbing.

3. Mo


INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 13:20:49] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 13:23:30] "GET /?question=Suggest_some_cleaning_products HTTP/1.1" 200 -


1. Colin Glass Cleaner - This product is designed to clean glass surfaces like windows, mirrors, and car windshields. It contains vinegar as the main ingredient which helps remove dirt and grime without leaving streaks or smears behind.   
2. Colin Multi-Purpose Cleaner - This product can be used on a variety of surfaces including countertops, appliances, floors, and furniture. It has a fresh citrus scent and is safe for


INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 13:23:31] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 13:26:06] "GET /?question=Suggest_something_to_eat HTTP/1.1" 200 -


1. Coconut Rice-Ready to Eat   250 GRM - This pouch contains cooked basmati rice with grated coconuts, curry leaves, green chilly & salt for ready to eat.

2. Ready To Eat - Navratan Kurma - Filled with carefully picked vegetables and dry fruits, a rich royal flavoured curry to delight your senses.

3. Tuna Tomato Rice - Ready to E


INFO:werkzeug:127.0.0.1 - - [30/Nov/2023 13:26:07] "GET /favicon.ico HTTP/1.1" 404 -
